# start_pakistan_correlations
## FL_calcTriggers.ipynb
This script takes the Pakistan flood event loss table (ELT) and applies the 1-in-3.5 years RP threshold to determine which events would trigger or not. We use the ELT simulation as many possible realisations of the history, in the absence of good historical data. We take abritrary 20 year periods and slide along a day each time, for the full 10,000 years in the ELT.

In [2]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt

In [4]:
# Set the root path
rootPath = Path('C:/Users/alexa/Documents/02_work/02_start/02_deliv/05_pk_correlation/fl/data')

## Define the threshold
1-in-3.5 years RP from the OEP loss curve, in number of people affected

In [19]:
thresh=1472522#.38102849

## Load the ELT data

In [6]:
elt=pd.read_csv(rootPath/'PortfolioLevel_0.2m_elt.csv')

In [7]:
elt

,event_ref,year,loss
0,R02_2016R1_Sim_00000066,1,16.077007
1,R02_2016R1_Sim_00000137,2,1.238141
2,R02_2016R1_Sim_00000258,3,96.624800
3,R02_2016R1_Sim_00001500,14,15.386976
4,R02_2016R1_Sim_00001933,17,17.317762
...,...,...,...
62793,R14_2017R1_Sim_00295097,9994,24.913216
62794,R14_2017R1_Sim_00295117,9995,2.008603
62795,R14_2017R1_Sim_00295122,9995,3187.793090
62796,R14_2017R1_Sim_00295226,9997,6.568177


In [20]:
print('{0} events out of a total {1} ({2:.2f}%) exceed the threshold, from {3} years of simulations.'
      .format(len(elt[elt.loss>=thresh]), len(elt), 
              len(elt[elt.loss>=thresh]) / len(elt) * 100, max(elt.year)))
print('Equates to an average of {0:.3f} events per year, or 1 event every {1:.2f} years.'
      .format(len(elt[elt.loss>=thresh]) / max(elt.year), max(elt.year) / len(elt[elt.loss>=thresh])))

3282 events out of a total 62798 (5.23%) exceed the threshold, from 10000 years of simulations.
Equates to an average of 0.328 events per year, or 1 event every 3.05 years.


## Iterate through the simulations in 20 year periods and record the triggers for each realiasation

In [22]:
elt=elt.sort_values('year').set_index('year')

In [23]:
elt

,event_ref,loss
year,,
1,R02_2016R1_Sim_00000066,1.607701e+01
1,R14_2017R1_Sim_00000009,9.665986e+00
1,R05_2016R1_Sim_00000018,2.012449e+00
1,R14_2017R1_Sim_00000002,1.047985e+00
2,R02_2016R1_Sim_00000137,1.238141e+00
...,...,...
10000,R05_2016R1_Sim_00581443,2.450462e+02
10000,R05_2016R1_Sim_00581444,5.073040e+05
10000,R05_2016R1_Sim_00581449,8.631921e+00


In [121]:
# Define no of years length period to use
nyears=19
lastStart = max(elt.index)-nyears+1
nrealisations=len(range(1, lastStart+1))

In [122]:
# Create dataframe for saving results
realisations = pd.DataFrame(data=[], index=list(range(1, nyears+1)))
nevents=[]

# Iterate through the entire elt, shifting the window by one year each time
for i, startYear in enumerate(range(1, lastStart+1)):

    # Get a subset nyear period of the elt
    eltPeriod = elt[np.logical_and(elt.index>=startYear, elt.index<(startYear+nyears))]
    
    # Get summary stats about the period - no of events
    nevents.append(len(eltPeriod))
    
    # Count no of triggers 
    eltPeriodTrigger = eltPeriod[eltPeriod.loss>=thresh]
    
    # Count triggers per year, then reset to a period from 1 to nyears
    yearTrigger = pd.DataFrame(eltPeriodTrigger.loss.groupby(by=eltPeriodTrigger.index).count())
    yearTrigger.index=yearTrigger.index-startYear+1
    
    # Record in the dataframe
    realisations = realisations.merge(yearTrigger, left_index=True, right_index=True, how='outer').fillna(0)
    
realisations.columns=list(range(1, lastStart+1))

C:\Users\alexa\AppData\Local\Temp\ipykernel_20452\4123811072.py:22: FutureWarning: Passing 'suffixes' which cause duplicate columns {'loss_x'} in the result is deprecated and will raise a MergeError in a future version.
  realisations = realisations.merge(yearTrigger, left_index=True, right_index=True, how='outer').fillna(0)
C:\Users\alexa\AppData\Local\Temp\ipykernel_20452\4123811072.py:22: FutureWarning: Passing 'suffixes' which cause duplicate columns {'loss_x'} in the result is deprecated and will raise a MergeError in a future version.
  realisations = realisations.merge(yearTrigger, left_index=True, right_index=True, how='outer').fillna(0)
C:\Users\alexa\AppData\Local\Temp\ipykernel_20452\4123811072.py:22: FutureWarning: Passing 'suffixes' which cause duplicate columns {'loss_x'} in the result is deprecated and will raise a MergeError in a future version.
  realisations = realisations.merge(yearTrigger, left_index=True, right_index=True, how='outer').fillna(0)
C:\Users\alexa\AppD

In [125]:
# Write out the realisations with no of triggers per year as csv
outPath=rootPath/'realisations'
outPath.mkdir(exist_ok=True)
realisations.T.to_csv(outPath/('realisationsTriggers_'+str(nyears)+'years.csv'), index=True)

### Cap triggers at max 1 per year

In [126]:
realisationsMax1 = realisations.copy(deep=True)
realisationsMax1[realisationsMax1>1]=1
realisationsMax1.T.to_csv(outPath/('realisationsTriggersMax1_'+str(nyears)+'years.csv'), index=True)

## Summarize the realisations
Number of events and number of triggers

In [ ]:
ntriggers=[]
maxtriggers=[]
for i, startYear in enumerate(range(1, lastStart+1)):
    ntriggers.append(realisations[startYear].sum())    
    maxtriggers.append(realisations[startYear].max()) 

In [127]:
# Summarize in dataframe
realisationsSummary = pd.DataFrame(data=[range(1, lastStart+1), nevents, ntriggers, maxtriggers], index=['realisation', 'NoEvents', 'NoTriggers', 'MaxTriggersYear']).T
realisationsSummary['TriggerPerYear'] = realisationsSummary.NoTriggers / nyears
realisationsSummary['YearsPerTrigger'] = nyears / realisationsSummary.NoTriggers
realisationsSummary.to_csv(outPath/('realisationsSummary_'+str(nyears)+'years.csv'), index=False)
realisationsSummary

,realisation,NoEvents,NoTriggers,MaxTriggersYear,TriggerPerYear,YearsPerTrigger
0,1.0,98.0,7.0,1.0,0.368421,2.714286
1,2.0,103.0,7.0,1.0,0.368421,2.714286
2,3.0,100.0,7.0,1.0,0.368421,2.714286
3,4.0,102.0,7.0,1.0,0.368421,2.714286
4,5.0,97.0,6.0,1.0,0.315789,3.166667
...,...,...,...,...,...,...
9977,9978.0,151.0,7.0,1.0,0.368421,2.714286
9978,9979.0,145.0,7.0,1.0,0.368421,2.714286
9979,9980.0,141.0,7.0,1.0,0.368421,2.714286
9980,9981.0,137.0,7.0,1.0,0.368421,2.714286


In [129]:
realisationsSummary.mean()

realisation        4991.500000
NoEvents            119.326387
NoTriggers            5.714157
MaxTriggersYear       1.000000
TriggerPerYear        0.300745
YearsPerTrigger       3.822540
dtype: float64

### Repeat for the realisations capped to max one trigger per year

In [ ]:
ntriggers=[]
maxtriggers=[]
for i, startYear in enumerate(range(1, lastStart+1)):
    ntriggers.append(realisationsMax1[startYear].sum())    
    maxtriggers.append(realisationsMax1[startYear].max())     

In [128]:
# Summarize in dataframe
realisationsMax1Summary = pd.DataFrame(data=[range(1, lastStart+1), nevents, ntriggers, maxtriggers], index=['realisation', 'NoEvents', 'NoTriggers', 'MaxTriggersYear']).T
realisationsMax1Summary['TriggerPerYear'] = realisationsMax1Summary.NoTriggers / nyears
realisationsMax1Summary['YearsPerTrigger'] = nyears / realisationsMax1Summary.NoTriggers
realisationsMax1Summary.to_csv(outPath/('realisationsMax1Summary_'+str(nyears)+'years.csv'), index=False)
realisationsMax1Summary

,realisation,NoEvents,NoTriggers,MaxTriggersYear,TriggerPerYear,YearsPerTrigger
0,1.0,98.0,7.0,1.0,0.368421,2.714286
1,2.0,103.0,7.0,1.0,0.368421,2.714286
2,3.0,100.0,7.0,1.0,0.368421,2.714286
3,4.0,102.0,7.0,1.0,0.368421,2.714286
4,5.0,97.0,6.0,1.0,0.315789,3.166667
...,...,...,...,...,...,...
9977,9978.0,151.0,7.0,1.0,0.368421,2.714286
9978,9979.0,145.0,7.0,1.0,0.368421,2.714286
9979,9980.0,141.0,7.0,1.0,0.368421,2.714286
9980,9981.0,137.0,7.0,1.0,0.368421,2.714286


In [130]:
realisationsMax1Summary.mean()

realisation        4991.500000
NoEvents            119.326387
NoTriggers            5.714157
MaxTriggersYear       1.000000
TriggerPerYear        0.300745
YearsPerTrigger       3.822540
dtype: float64

In [131]:
nyears

19